# Day 8_0: 시계열 기초 - 정답 노트북

## 📚 학습 목표

**Part 1: 기초**
1. 시계열 데이터의 개념과 특성 이해
2. `pd.to_datetime()`으로 날짜 문자열 변환
3. `DatetimeIndex` 설정 및 활용
4. 시간대(timezone) 처리 방법
5. 날짜 범위 생성 (`date_range`)

**Part 2: 심화**
1. 시계열 인덱싱 및 슬라이싱 마스터
2. `shift()`로 시차 데이터 생성
3. `diff()`로 변화량 계산
4. `px.line()`으로 시계열 시각화

---

## 🎯 실습 퀴즈 정답

---

### Q1. 날짜 문자열 변환 ⭐

**문제**: 다음 날짜 문자열 리스트를 datetime 객체로 변환하세요. 변환 실패 시 NaT로 처리하세요.

In [ ]:
# Q1 정답
import pandas as pd
import numpy as np

date_strings = ['2024-01-15', '2024/02/20', 'invalid_date', '2024-03-25']

# errors='coerce'로 변환 실패 시 NaT 처리
converted_dates = pd.to_datetime(date_strings, errors='coerce')

print("변환 결과:")
print(converted_dates)
print(f"\nNaT 개수: {converted_dates.isna().sum()}개")

In [ ]:
# 테스트
assert len(converted_dates) == 4, "결과 개수가 4개여야 합니다"
assert converted_dates.isna().sum() == 1, "NaT가 1개 있어야 합니다"
assert converted_dates[0] == pd.Timestamp('2024-01-15'), "첫 번째 날짜 변환 오류"
print("✅ 테스트 통과!")

### 💡 Q1 풀이 설명

**접근 방법**:
`pd.to_datetime()`의 `errors` 파라미터를 사용하여 변환 실패 시 프로그램이 중단되지 않도록 처리합니다.

**핵심 개념**:
- `errors='raise'`: 기본값, 실패 시 에러 발생
- `errors='coerce'`: 실패 시 NaT (Not a Time) 반환
- `errors='ignore'`: 실패 시 원본 값 유지

**대안 솔루션**:
```python
# 리스트 컴프리헨션 + try-except
result = []
for date_str in date_strings:
    try:
        result.append(pd.to_datetime(date_str))
    except:
        result.append(pd.NaT)
```

**흔한 실수**:
- ❌ `errors` 파라미터 누락으로 에러 발생
- ❌ NaT 개수 확인하지 않고 넘어가기

**실무 팁**:
웹 스크레이핑 데이터는 날짜 형식이 불규칙하므로 항상 `errors='coerce'` 사용 후 `.isna().sum()`으로 변환 실패 개수를 확인하세요.

---

### Q2. DatetimeIndex 설정 ⭐⭐

**문제**: DataFrame의 `date` 컬럼을 DatetimeIndex로 설정하고, 인덱스의 최소/최대 날짜를 출력하세요.

In [ ]:
# Q2 정답
data = {
    'date': ['2024-01-01', '2024-01-05', '2024-01-10', '2024-01-15'],
    'sales': [1200, 1500, 1350, 1600]
}
df_q2 = pd.DataFrame(data)

# 1. date 컬럼을 datetime으로 변환
df_q2['date'] = pd.to_datetime(df_q2['date'])

# 2. DatetimeIndex로 설정
df_q2 = df_q2.set_index('date')

# 3. 최소/최대 날짜 출력
print(f"최소 날짜: {df_q2.index.min()}")
print(f"최대 날짜: {df_q2.index.max()}")
print(f"\n데이터 기간: {(df_q2.index.max() - df_q2.index.min()).days}일")

In [ ]:
# 테스트
assert isinstance(df_q2.index, pd.DatetimeIndex), "DatetimeIndex여야 합니다"
assert df_q2.index.min() == pd.Timestamp('2024-01-01'), "최소 날짜 오류"
assert df_q2.index.max() == pd.Timestamp('2024-01-15'), "최대 날짜 오류"
print("✅ 테스트 통과!")

### 💡 Q2 풀이 설명

**접근 방법**:
1. 문자열 날짜를 datetime으로 변환
2. `set_index()`로 해당 컬럼을 인덱스로 설정
3. 인덱스의 `.min()`, `.max()` 메서드 활용

**핵심 개념**:
- DatetimeIndex는 날짜 관련 속성/메서드 제공 (`.min()`, `.max()`, `.year`, `.month` 등)
- `set_index()`는 원본 변경 안 함 → 재할당 필요

**대안 솔루션**:
```python
# 한 번에 처리
df_q2 = pd.DataFrame(data)
df_q2.index = pd.to_datetime(df_q2.pop('date'))
```

**흔한 실수**:
- ❌ `set_index()` 결과를 재할당 안 함 (원본 변경 안 됨)
- ❌ datetime 변환 전에 바로 `set_index()` 시도

**실무 팁**:
CSV 읽을 때 `pd.read_csv(..., parse_dates=['date'], index_col='date')`로 한번에 처리 가능합니다.

---

### Q3. 영업일 날짜 범위 생성 ⭐⭐

**문제**: 2024년 1월 1일부터 20개의 영업일(주말 제외) 날짜를 생성하고, 첫 5개와 마지막 5개 날짜를 출력하세요.

In [ ]:
# Q3 정답

# freq='B' → Business day (영업일)
business_dates = pd.date_range(start='2024-01-01', periods=20, freq='B')

print("📅 영업일 20개 생성")
print(f"총 개수: {len(business_dates)}\n")

print("처음 5개:")
print(business_dates[:5])

print("\n마지막 5개:")
print(business_dates[-5:])

In [ ]:
# 테스트: 주말 포함 확인
weekdays = business_dates.dayofweek  # 0=월, 6=일
assert all(weekdays < 5), "주말(토/일)이 포함되어 있습니다"
assert len(business_dates) == 20, "20개가 아닙니다"
print("✅ 테스트 통과! (주말 제외 확인)")

### 💡 Q3 풀이 설명

**접근 방법**:
`pd.date_range()`의 `freq='B'` 파라미터로 영업일만 자동 생성합니다.

**핵심 개념**:
- `freq='D'`: 모든 날 (All days)
- `freq='B'`: 영업일 (Business days, 월-금)
- `freq='W'`: 주간 (Weekly)

**대안 솔루션**:
```python
# 수동으로 주말 필터링
all_dates = pd.date_range(start='2024-01-01', periods=30, freq='D')
business_only = all_dates[all_dates.dayofweek < 5][:20]
```

**흔한 실수**:
- ❌ `freq='D'` 사용 후 수동 필터링 (비효율)
- ❌ 공휴일도 제외된다고 착각 (공휴일은 포함됨)

**실무 팁**:
한국 공휴일까지 제외하려면 `pandas-market-calendars` 또는 `workalendar` 라이브러리를 사용하세요.

---

### Q4. 시간대 변환 ⭐⭐

**문제**: 서울 시간 `2024-01-15 14:00:00`을 뉴욕 시간(America/New_York)으로 변환하세요.

In [ ]:
# Q4 정답

# 1. naive 시간 생성
naive_time = pd.Timestamp('2024-01-15 14:00:00')
print(f"Naive 시간: {naive_time}")

# 2. 서울 시간대 지정
seoul_time = naive_time.tz_localize('Asia/Seoul')
print(f"Seoul 시간: {seoul_time}")

# 3. 뉴욕 시간대로 변환
ny_time = seoul_time.tz_convert('America/New_York')
print(f"뉴욕 시간: {ny_time}")

print(f"\n시차: {seoul_time.hour - ny_time.hour}시간")

In [ ]:
# 테스트
assert seoul_time.tz is not None, "시간대 정보가 없습니다"
assert ny_time.tz.zone == 'America/New_York', "뉴욕 시간대가 아닙니다"
assert ny_time.hour == 0, "변환 오류 (서울 14시 = 뉴욕 0시)"
print("✅ 테스트 통과!")

### 💡 Q4 풀이 설명

**접근 방법**:
1. `tz_localize()`: naive → aware (시간대 정보 추가)
2. `tz_convert()`: aware → aware (다른 시간대로 변환)

**핵심 개념**:
- Naive 시간: 시간대 정보 없음 (`tz=None`)
- Aware 시간: 시간대 정보 있음 (`tz='Asia/Seoul'` 등)
- 서울-뉴욕 시차: 약 14시간 (서울이 앞섬)

**대안 솔루션**:
```python
# 한 번에 처리
ny_time = pd.Timestamp('2024-01-15 14:00:00', tz='Asia/Seoul').tz_convert('America/New_York')
```

**흔한 실수**:
- ❌ naive 시간에 바로 `tz_convert()` 시도 (에러)
- ❌ `tz_localize()` vs `tz_convert()` 혼동

**실무 팁**:
글로벌 서비스에서는 모든 시간을 UTC로 저장하고, 사용자 시간대에 맞춰 표시할 때만 `tz_convert()` 사용을 권장합니다.

---

### Q5. 특정 월 데이터 필터링 ⭐⭐⭐

**문제**: DataFrame에서 2024년 3월 데이터만 추출하고, 해당 월의 평균 매출을 계산하세요.

In [ ]:
# Q5 정답
np.random.seed(42)
dates = pd.date_range(start='2024-01-01', end='2024-06-30', freq='D')
df_q5 = pd.DataFrame({
    'revenue': np.random.randint(1000, 2000, len(dates))
}, index=dates)

# 3월 데이터 추출 (부분 문자열 인덱싱)
march_data = df_q5['2024-03']

print("📅 2024년 3월 데이터:")
print(f"데이터 개수: {len(march_data)}개")
print(f"기간: {march_data.index.min()} ~ {march_data.index.max()}")
print(f"\n3월 평균 매출: {march_data['revenue'].mean():.0f}만원")
print(f"3월 총 매출: {march_data['revenue'].sum():,}만원")

In [ ]:
# 테스트
assert len(march_data) == 31, "3월은 31일입니다"
assert march_data.index.min().month == 3, "3월 데이터가 아닙니다"
assert march_data.index.max().month == 3, "3월 데이터가 아닙니다"
print("✅ 테스트 통과!")

### 💡 Q5 풀이 설명

**접근 방법**:
DatetimeIndex의 부분 문자열 매칭 기능을 활용하여 `df['2024-03']`로 3월 전체 데이터를 한번에 추출합니다.

**핵심 개념**:
- `df['2024']`: 2024년 전체
- `df['2024-03']`: 2024년 3월 전체
- `df['2024-03-15']`: 특정 날짜
- `df['2024-Q1']`: 1분기 (연도-분기)

**대안 솔루션**:
```python
# 조건 필터링
march_data = df_q5[df_q5.index.month == 3]

# 범위 슬라이싱
march_data = df_q5['2024-03-01':'2024-03-31']
```

**흔한 실수**:
- ❌ `df_q5[df_q5.index == '2024-03']` (문자열 비교 오류)
- ❌ 인덱스가 DatetimeIndex가 아닐 때 부분 문자열 매칭 시도

**실무 팁**:
분기별 집계는 `df['2024-Q1']` 형태로, 반기별은 `df['2024-01':'2024-06']` 슬라이싱을 사용하세요.

---

### Q6. shift()로 전일 매출 비교 ⭐⭐⭐

**문제**: DataFrame에 전일 매출(`prev_revenue`)과 전일 대비 증가액(`change`) 컬럼을 추가하세요.

In [ ]:
# Q6 정답
dates = pd.date_range(start='2024-01-01', periods=7, freq='D')
df_q6 = pd.DataFrame({
    'revenue': [1200, 1350, 1180, 1420, 1590, 1500, 1650]
}, index=dates)

# shift(1)로 전일 매출 생성
df_q6['prev_revenue'] = df_q6['revenue'].shift(1)

# 전일 대비 증가액
df_q6['change'] = df_q6['revenue'] - df_q6['prev_revenue']

print("📊 전일 매출 비교:")
print(df_q6)

# 통계
print(f"\n평균 일일 변화: {df_q6['change'].mean():.0f}만원")
print(f"최대 상승: {df_q6['change'].max():.0f}만원")
print(f"최대 하락: {df_q6['change'].min():.0f}만원")

In [ ]:
# 테스트
assert 'prev_revenue' in df_q6.columns, "prev_revenue 컬럼 없음"
assert 'change' in df_q6.columns, "change 컬럼 없음"
assert pd.isna(df_q6.iloc[0]['prev_revenue']), "첫 번째 행의 prev_revenue는 NaN이어야 함"
assert df_q6.iloc[1]['change'] == 150, "두 번째 행의 change 계산 오류"
print("✅ 테스트 통과!")

### 💡 Q6 풀이 설명

**접근 방법**:
1. `shift(1)`: 데이터를 1행 아래로 이동 → 전일 값
2. 현재 값 - 전일 값 = 변화량

**핵심 개념**:
- `shift(1)`: 1기간 뒤로 (과거 값)
- `shift(-1)`: 1기간 앞으로 (미래 값)
- 첫 번째 행의 `shift(1)` 결과는 NaN

**대안 솔루션**:
```python
# diff()를 직접 사용
df_q6['change'] = df_q6['revenue'].diff()
# 하지만 prev_revenue는 shift()로만 생성 가능
```

**흔한 실수**:
- ❌ `shift(-1)` 사용 (다음날 값과 비교됨)
- ❌ NaN 처리 안 함 (첫 행 계산 오류)

**실무 팁**:
주식 데이터 분석 시 `pct_change()` 메서드를 사용하면 변화율(%)을 바로 계산할 수 있습니다: `df['return'] = df['close'].pct_change()`

---

### Q7. diff()로 일일 신규 가입자 계산 ⭐⭐⭐

**문제**: 누적 회원 수에서 일일 신규 가입자를 계산하고, 가장 많은 신규 가입자가 발생한 날짜와 수를 출력하세요.

In [ ]:
# Q7 정답
dates = pd.date_range(start='2024-01-01', periods=10, freq='D')
df_q7 = pd.DataFrame({
    'total_members': [5000, 5025, 5060, 5095, 5110, 5150, 5185, 5200, 5240, 5275]
}, index=dates)

# diff()로 일일 신규 가입자 계산
df_q7['new_members'] = df_q7['total_members'].diff()

print("👥 일일 신규 가입자:")
print(df_q7)

# 최대 신규 가입일 찾기
max_idx = df_q7['new_members'].idxmax()
max_value = df_q7['new_members'].max()

print(f"\n최대 신규 가입일: {max_idx.date()}")
print(f"신규 가입자 수: {max_value:.0f}명")

In [ ]:
# 테스트
assert 'new_members' in df_q7.columns, "new_members 컬럼 없음"
assert df_q7['new_members'].max() == 40, "최대 신규 가입자 계산 오류"
assert max_idx == pd.Timestamp('2024-01-06'), "최대 가입일 오류"
print("✅ 테스트 통과!")

### 💡 Q7 풀이 설명

**접근 방법**:
1. `diff()`: 연속된 값의 차이 계산
2. `idxmax()`: 최대값의 인덱스(날짜) 반환
3. `max()`: 최대값 자체 반환

**핵심 개념**:
- 누적 데이터 → `diff()` → 일별 변화량
- `diff()` = 현재 값 - 이전 값
- 첫 번째 행은 NaN (비교 대상 없음)

**대안 솔루션**:
```python
# shift() 사용
df_q7['new_members'] = df_q7['total_members'] - df_q7['total_members'].shift(1)
```

**흔한 실수**:
- ❌ `idxmax()` vs `max()` 혼동 (인덱스 vs 값)
- ❌ NaN 행 포함하여 통계 계산 (`.dropna()` 필요할 수도)

**실무 팁**:
구글 애널리틱스 등에서 누적 지표를 받았을 때, `diff()`로 일별/월별 증가량을 계산하여 실제 성장률을 분석하세요.

---

### Q8. 주말/평일 매출 비교 ⭐⭐⭐⭐

**문제**: 2024년 1월 전체 데이터에서 주말(토/일) 평균 매출과 평일 평균 매출을 계산하고, 차이를 출력하세요.

In [ ]:
# Q8 정답
np.random.seed(42)
dates = pd.date_range(start='2024-01-01', end='2024-01-31', freq='D')
df_q8 = pd.DataFrame({
    'revenue': np.random.randint(1000, 2500, len(dates))
}, index=dates)

# dayofweek: 0=월요일, 6=일요일
# 주말: 5(토), 6(일)
weekend_mask = df_q8.index.dayofweek.isin([5, 6])
weekday_mask = ~weekend_mask  # not 연산자

# 평균 계산
weekend_avg = df_q8[weekend_mask]['revenue'].mean()
weekday_avg = df_q8[weekday_mask]['revenue'].mean()
difference = weekend_avg - weekday_avg

print("📊 주말 vs 평일 매출 비교:")
print(f"평일 평균: {weekday_avg:.0f}만원 ({weekday_mask.sum()}일)")
print(f"주말 평균: {weekend_avg:.0f}만원 ({weekend_mask.sum()}일)")
print(f"차이: {difference:.0f}만원")

if difference > 0:
    print(f"\n✅ 주말이 평일보다 {difference:.0f}만원 높음")
else:
    print(f"\n⚠️ 평일이 주말보다 {-difference:.0f}만원 높음")

In [ ]:
# 테스트
assert weekday_mask.sum() == 23, "1월 평일은 23일입니다"
assert weekend_mask.sum() == 8, "1월 주말은 8일입니다"
assert isinstance(weekday_avg, (int, float)), "평균값이 숫자여야 합니다"
print("✅ 테스트 통과!")

### 💡 Q8 풀이 설명

**접근 방법**:
1. `index.dayofweek`로 요일 정보 추출 (0=월, 6=일)
2. `.isin([5, 6])`로 주말 마스크 생성
3. 마스크로 데이터 필터링 후 평균 계산

**핵심 개념**:
- `dayofweek`: 0(월) ~ 6(일)
- `day_name()`: 'Monday', 'Tuesday', ...
- Boolean 마스킹으로 조건부 집계

**대안 솔루션**:
```python
# groupby 사용
df_q8['is_weekend'] = df_q8.index.dayofweek.isin([5, 6])
result = df_q8.groupby('is_weekend')['revenue'].mean()
```

**흔한 실수**:
- ❌ `dayofweek == 6` (일요일만 선택됨)
- ❌ `isin(['토', '일'])` (문자열 X, 숫자 사용)

**실무 팁**:
리테일 분석에서 주말/평일 패턴 파악은 필수입니다. 시간대별 분석도 `index.hour`로 가능합니다.

---

### Q9. px.line() 멀티 시리즈 시각화 ⭐⭐⭐⭐

**문제**: DataFrame의 `매출`, `비용`, `순이익`을 하나의 그래프에 선으로 표시하세요.

In [ ]:
# Q9 정답
import plotly.graph_objects as go

months = pd.date_range(start='2024-01', periods=6, freq='MS')
df_q9 = pd.DataFrame({
    '매출': [5000, 5500, 6000, 6200, 6800, 7200],
    '비용': [3000, 3200, 3500, 3600, 4000, 4200],
    '순이익': [2000, 2300, 2500, 2600, 2800, 3000]
}, index=months)

# plotly graph_objects로 멀티 시리즈 차트
fig = go.Figure()

# 각 컬럼을 별도 trace로 추가
for col in df_q9.columns:
    fig.add_trace(go.Scatter(
        x=df_q9.index,
        y=df_q9[col],
        mode='lines+markers',
        name=col,
        line=dict(width=3)
    ))

fig.update_layout(
    title='📊 상반기 재무 현황',
    xaxis_title='월',
    yaxis_title='금액 (만원)',
    hovermode='x unified',
    height=500,
    legend=dict(x=0.01, y=0.99)
)

fig.show()

In [ ]:
# 테스트
assert len(fig.data) == 3, "3개의 시리즈가 있어야 합니다"
assert all(trace.mode == 'lines+markers' for trace in fig.data), "선+마커 모드여야 합니다"
print("✅ 테스트 통과!")

### 💡 Q9 풀이 설명

**접근 방법**:
1. `go.Figure()` 생성
2. 각 컬럼을 순회하며 `add_trace()` 추가
3. `update_layout()`으로 스타일 설정

**핵심 개념**:
- `go.Scatter`: 선/점 그래프
- `mode='lines+markers'`: 선과 마커 동시 표시
- `hovermode='x unified'`: 같은 x축 값 통합 표시

**대안 솔루션**:
```python
# px.line 사용 (wide format → long format 변환 필요)
df_long = df_q9.reset_index().melt(id_vars='index', var_name='구분', value_name='금액')
fig = px.line(df_long, x='index', y='금액', color='구분', markers=True)
```

**흔한 실수**:
- ❌ 각 시리즈를 별도 그래프로 생성 (비교 어려움)
- ❌ 범례 누락으로 어떤 선인지 구분 불가

**실무 팁**:
경영진 보고용 차트는 `hovermode='x unified'`로 마우스 오버 시 모든 값을 한번에 보여주는 것이 효과적입니다.

---

### Q10. 종합 분석: 매출 전월 대비 증가율 ⭐⭐⭐⭐⭐

**문제**: 2024년 월별 매출 데이터에서:
1. 전월 대비 증가율(%) 계산
2. 증가율이 가장 높은 달 찾기
3. 매출과 증가율을 하나의 차트에 시각화 (2축)

In [ ]:
# Q10 정답
from plotly.subplots import make_subplots

months = pd.date_range(start='2024-01', periods=12, freq='MS')
df_q10 = pd.DataFrame({
    'revenue': [5000, 5200, 5800, 6100, 6500, 7200, 7800, 7600, 7400, 7800, 8500, 9200]
}, index=months)

# 1. 전월 대비 증가율 계산
df_q10['prev_revenue'] = df_q10['revenue'].shift(1)
df_q10['growth_rate'] = ((df_q10['revenue'] - df_q10['prev_revenue']) / df_q10['prev_revenue'] * 100).round(2)

print("📊 월별 매출 및 증가율:")
print(df_q10[['revenue', 'growth_rate']])

# 2. 최고 증가율 월 찾기
max_growth_month = df_q10['growth_rate'].idxmax()
max_growth_value = df_q10['growth_rate'].max()

print(f"\n최고 증가율 월: {max_growth_month.strftime('%Y-%m')}")
print(f"증가율: {max_growth_value:.2f}%")

In [ ]:
# 3. 2축 차트 시각화
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 매출 (막대)
fig.add_trace(
    go.Bar(
        x=df_q10.index,
        y=df_q10['revenue'],
        name='매출',
        marker_color='lightblue',
        opacity=0.7
    ),
    secondary_y=False
)

# 증가율 (선)
fig.add_trace(
    go.Scatter(
        x=df_q10.index,
        y=df_q10['growth_rate'],
        name='전월 대비 증가율',
        mode='lines+markers',
        line=dict(color='red', width=3),
        marker=dict(size=8)
    ),
    secondary_y=True
)

# 레이아웃 설정
fig.update_layout(
    title='📈 2024년 월별 매출 및 증가율',
    xaxis_title='월',
    hovermode='x unified',
    height=500
)

fig.update_yaxes(title_text="매출 (만원)", secondary_y=False)
fig.update_yaxes(title_text="증가율 (%)", secondary_y=True)

fig.show()

In [ ]:
# 테스트
assert 'growth_rate' in df_q10.columns, "growth_rate 컬럼 없음"
assert max_growth_month == pd.Timestamp('2024-07-01'), "최고 증가율 월 오류"
assert abs(max_growth_value - 8.33) < 0.1, "증가율 계산 오류"
print("✅ 테스트 통과!")

### 💡 Q10 풀이 설명

**접근 방법**:
1. `shift(1)`로 전월 매출 생성
2. 증가율 공식: (현재 - 이전) / 이전 × 100
3. `make_subplots()`로 2축 차트 생성

**핵심 개념**:
- 증가율: (변화량 / 기준값) × 100
- 2축 차트: 스케일이 다른 데이터 동시 표시
- `secondary_y=True`: 오른쪽 y축 사용

**대안 솔루션**:
```python
# pct_change() 사용
df_q10['growth_rate'] = df_q10['revenue'].pct_change() * 100
```

**흔한 실수**:
- ❌ 증가액과 증가율 혼동
- ❌ 2축 차트에서 y축 라벨 누락 (혼란)
- ❌ 백분율 변환 안 함 (0.0833 → 8.33%)

**실무 팁**:
경영 대시보드에서 절대값(매출)과 상대값(증가율)을 함께 보여주면 트렌드와 변화 속도를 동시에 파악할 수 있습니다. 특히 7월 매출 급증 원인(프로모션, 계절성 등)을 추가 분석할 수 있습니다.

---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메소드/문법 | 실무 활용 |
|------|-----------------|----------|
| 시계열 데이터 | 시간 순서, 의존성 | 주가, 매출, 트래픽 분석 |
| datetime 변환 | `pd.to_datetime()` | 문자열 날짜 → 분석 가능 객체 |
| DatetimeIndex | `df.set_index()` | 날짜 기반 필터링/슬라이싱 |
| 시간대 처리 | `tz_localize()`, `tz_convert()` | 글로벌 서비스 로그 분석 |
| 날짜 범위 생성 | `pd.date_range()` | 월별/일별 템플릿 생성 |

### Part 2: 심화 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| 시계열 인덱싱 | `df['2024-01']`, `df['2024-01':'2024-03']` | 특정 기간 데이터 추출 |
| shift() | `df['col'].shift(1)` | 전일/전월 값 비교 |
| diff() | `df['col'].diff()` | 변화량, 증가폭 계산 |
| px.line() | `px.line(df, x, y, markers=True)` | 시계열 추세 시각화 |

### 💡 실무 팁

1. **날짜 파싱 오류 대비**: `errors='coerce'`로 잘못된 날짜 NaT 처리 후 확인
2. **시간대 일관성**: 모든 데이터를 UTC로 통일 후 필요시 로컬 시간으로 변환
3. **영업일 계산**: `freq='B'`로 주말 자동 제외 (공휴일은 별도 처리 필요)
4. **shift vs diff 선택**: 원본 값 필요 시 `shift()`, 변화량만 필요 시 `diff()`
5. **시각화 성능**: 데이터가 많으면 월별 집계 후 시각화로 속도 개선
6. **인덱스 리셋**: Plotly에 넘길 때 `reset_index()`로 날짜를 컬럼으로 변환
7. **이동평균 활용**: 노이즈 많은 일별 데이터는 7일/30일 이동평균으로 추세 파악
8. **2축 차트**: 절대값과 상대값(증가율) 동시 표시로 인사이트 강화
9. **주말/평일 분석**: 리테일 데이터는 요일별 패턴 필수 확인
10. **증가율 계산**: `pct_change()` 메서드로 코드 간소화 가능

---

**다음 학습**: Day 8_1에서 `resample()` (리샘플링), `rolling()` (이동평균), 시계열 분해를 배웁니다! 🚀